元データを整形して'./tmp/transformed'へ保存

Data: https://archive.ics.uci.edu/dataset/447/condition+monitoring+of+hydraulic+systems

***

- 2205個のオブザベーション
- 17種類のセンサー（PS1,..,SE）
- 元データにおいて、各センサー値は（2205オブザベーション × 時系列）で格納されている

***

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

import pickle

In [2]:
!ls tmp/archive

CE.txt            PS1.txt           PS6.txt           TS4.txt
CP.txt            PS2.txt           SE.txt            VS1.txt
EPS1.txt          PS3.txt           TS1.txt           description.txt
FS1.txt           PS4.txt           TS2.txt           documentation.txt
FS2.txt           PS5.txt           TS3.txt           profile.txt


In [3]:
read_dir = 'tmp/archive/'
write_dir = 'tmp/transformed/'

### Target
profile.txt

In [4]:
target_cols = ['Cooler_Condition_pct', 'Valve_Condition_pct', 'Internal_Pump_Leakage', 'Hydraulic_Accumulato', 'Stable_flag']

df_profile = pd.read_csv(read_dir+'profile.txt', sep='\t', header=None, names=target_cols)
print(df_profile.shape)
df_profile.head()

(2205, 5)


,Cooler_Condition_pct,Valve_Condition_pct,Internal_Pump_Leakage,Hydraulic_Accumulato,Stable_flag
0,3,100,0,130,1
1,3,100,0,130,1
2,3,100,0,130,1
3,3,100,0,130,1
4,3,100,0,130,1


In [5]:
for c in target_cols:
    print('>>> ', c)
    print(df_profile[c].value_counts())

>>>  Cooler_Condition_pct
100    741
3      732
20     732
Name: Cooler_Condition_pct, dtype: int64
>>>  Valve_Condition_pct
100    1125
73      360
80      360
90      360
Name: Valve_Condition_pct, dtype: int64
>>>  Internal_Pump_Leakage
0    1221
2     492
1     492
Name: Internal_Pump_Leakage, dtype: int64
>>>  Hydraulic_Accumulato
90     808
130    599
115    399
100    399
Name: Hydraulic_Accumulato, dtype: int64
>>>  Stable_flag
0    1449
1     756
Name: Stable_flag, dtype: int64


In [6]:
df_profile = df_profile.reset_index(names=['id'])

In [7]:
df_profile.to_csv(write_dir+'profile.csv', index=False)

In [19]:
train, test = train_test_split(df_profile, test_size=300, stratify=df_profile['Stable_flag'],  random_state=123)
train.shape, test.shape

((1905, 6), (300, 6))

In [22]:
train['test_flg'] = 0
test['test_flg'] = 1

/var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/ipykernel_96020/3630869361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['test_flg'] = 0
/var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/ipykernel_96020/3630869361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['test_flg'] = 1


In [26]:
df_profile_tf = pd.concat([train, test])
df_profile_tf = df_profile_tf.sort_values('id')
df_profile_tf.shape

(2205, 7)

In [27]:
df_profile_tf.to_csv(write_dir+'profile_with_testflag.csv', index=False)

### Features

縦持ちデータへの変換

In [48]:
feature_sets = ['PS1','PS2','PS3','PS4','PS5','PS6','EPS1',   # 100 Hz
                'FS1','FS2',                                                                    # 10 Hz
                'TS1','TS2','TS3','TS4','VS1','CE','CP','SE']             # 1 Hz
feature_dfs = {}

for d in feature_sets:
    print(d)
    tmp = pd.read_csv(read_dir+d+'.txt', sep='\t', header=None)
    tmp2 = tmp.stack().reset_index()  # 縦持ち変換
    tmp2.columns = ['id', 'time', d]
    print(tmp.shape, tmp2.shape)
    feature_dfs[d] = tmp2

PS1
(2205, 6000) (13230000, 3)
PS2
(2205, 6000) (13230000, 3)
PS3
(2205, 6000) (13230000, 3)
PS4
(2205, 6000) (13230000, 3)
PS5
(2205, 6000) (13230000, 3)
PS6
(2205, 6000) (13230000, 3)
EPS1
(2205, 6000) (13230000, 3)
FS1
(2205, 600) (1323000, 3)
FS2
(2205, 600) (1323000, 3)
TS1
(2205, 60) (132300, 3)
TS2
(2205, 60) (132300, 3)
TS3
(2205, 60) (132300, 3)
TS4
(2205, 60) (132300, 3)
VS1
(2205, 60) (132300, 3)
CE
(2205, 60) (132300, 3)
CP
(2205, 60) (132300, 3)
SE
(2205, 60) (132300, 3)


In [53]:
len(feature_dfs)

17

In [70]:
for k in feature_sets:
    print(k)
    print('N of null', feature_dfs[k].isna().sum())

PS1
N of null id      0
time    0
PS1     0
dtype: int64
PS2
N of null id      0
time    0
PS2     0
dtype: int64
PS3
N of null id      0
time    0
PS3     0
dtype: int64
PS4
N of null id      0
time    0
PS4     0
dtype: int64
PS5
N of null id      0
time    0
PS5     0
dtype: int64
PS6
N of null id      0
time    0
PS6     0
dtype: int64
EPS1
N of null id      0
time    0
EPS1    0
dtype: int64
FS1
N of null id      0
time    0
FS1     0
dtype: int64
FS2
N of null id      0
time    0
FS2     0
dtype: int64
TS1
N of null id      0
time    0
TS1     0
dtype: int64
TS2
N of null id      0
time    0
TS2     0
dtype: int64
TS3
N of null id      0
time    0
TS3     0
dtype: int64
TS4
N of null id      0
time    0
TS4     0
dtype: int64
VS1
N of null id      0
time    0
VS1     0
dtype: int64
CE
N of null id      0
time    0
CE      0
dtype: int64
CP
N of null id      0
time    0
CP      0
dtype: int64
SE
N of null id      0
time    0
SE      0
dtype: int64


In [61]:
# to write
with open(write_dir+'features.pickle', mode='wb') as fo:
    pickle.dump(feature_dfs, fo)

In [62]:
# to read
with open(write_dir+'features.pickle', mode='br') as fi:
    read_data = pickle.load(fi)

In [63]:
len(read_data)

17

In [65]:
read_data.keys()

dict_keys(['PS1', 'PS2', 'PS3', 'PS4', 'PS5', 'PS6', 'EPS1', 'FS1', 'FS2', 'TS1', 'TS2', 'TS3', 'TS4', 'VS1', 'CE', 'CP', 'SE'])